In [ ]:
import os
!pip install ultralytics --quiet
from ultralytics import YOLO
!pip install kagglehub --quiet
import kagglehub
import cv2
!pip install tqdm
!pip install pillow
from tqdm import tqdm
from PIL import Image
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline


dataset_path = "/content/HRSC2016-MS/"

In [ ]:
def download_and_extract_dataset():
    print("Downloading HRSC2016-MS dataset from Kaggle Hub...")
    path = kagglehub.dataset_download("weiming97/hrsc2016-ms-dataset")
    print("Path to dataset files:", path)

    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
        os.system(f"cp -r {path}/* {dataset_path}")

In [ ]:
download_and_extract_dataset()

100%|██████████| 2.34G/2.34G [00:21<00:00, 115MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/weiming97/hrsc2016-ms-dataset/versions/1


In [ ]:
def make_yolo_dir(cwd):
    yolo_dir = os.path.join(cwd, 'HRSC-YOLO')
    os.mkdir(yolo_dir)
    for i in ['train', 'val', 'test']:
        folder = os.path.join(yolo_dir, i)
        os.mkdir(folder)
        for j in ['images', 'labels']:
            subfolder = os.path.join(folder, j)
            os.mkdir(subfolder)
    return yolo_dir

def xml2txt(xml_file_path, txt_file, w, h):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    objs = root.findall('object')

    if not objs:
        print(f"Warning: No objects found in {xml_file_path}. Creating an empty annotation file.")
        with open(txt_file, 'w') as f:
            f.write("\n")  # Empty file for compatibility
        return

    with open(txt_file, 'w') as f:
        for obj in objs:
            try:
                xmin = int(obj.find('bndbox/xmin').text)
                ymin = int(obj.find('bndbox/ymin').text)
                xmax = int(obj.find('bndbox/xmax').text)
                ymax = int(obj.find('bndbox/ymax').text)

                # Convert to YOLO format (normalized values)
                x_center = (xmin + xmax) / (2.0 * w)
                y_center = (ymin + ymax) / (2.0 * h)
                box_w = (xmax - xmin) / w
                box_h = (ymax - ymin) / h

                # Save in YOLO format
                f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
            except Exception as e:
                print(f"Error processing {xml_file_path}: {e}")



def read_list_file(cwd):
    with open(os.path.join(cwd, 'ImageSets/train.txt'), 'r') as f:
        train_list = f.read().splitlines()
    with open(os.path.join(cwd, 'ImageSets/val.txt'), 'r') as f:
        val_list = f.read().splitlines()
    with open(os.path.join(cwd, 'ImageSets/test.txt'), 'r') as f:
        test_list = f.read().splitlines()
    return train_list, val_list, test_list

def construct_path(file_name, cwd, yolo_dir, mode):
    if mode == 'train':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'train/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'train/images/{file_name}.png')
    elif mode == 'val':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'val/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'val/images/{file_name}.png')
    elif mode == 'test':
        train_file_path = os.path.join(cwd, f'AllImages/{file_name}.bmp')
        train_xml_path = os.path.join(cwd, f'Annotations/{file_name}.xml')
        save_txt_path = os.path.join(yolo_dir, f'test/labels/{file_name}.txt')
        save_png_path = os.path.join(yolo_dir, f'test/images/{file_name}.png')
    else:
        print(f"Unrecognized mode {mode}!")
    return train_file_path, train_xml_path, save_txt_path, save_png_path


def main():
    cwd = os.getcwd()
    yolo_dir = make_yolo_dir(cwd)

    train_list, val_list, test_list = read_list_file("/content/HRSC2016-MS")

    for train_file in tqdm(train_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(train_file, "/content/HRSC2016-MS", yolo_dir, 'train')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

    for val_file in tqdm(val_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(val_file, "/content/HRSC2016-MS", yolo_dir, 'val')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

    for test_file in tqdm(test_list):
        bmp_path, xml_path, txt_path, png_path = construct_path(test_file, "/content/HRSC2016-MS", yolo_dir, 'test')

        img = Image.open(bmp_path)
        w, h = img.size
        xml2txt(xml_path, txt_path, w, h)

        img.save(png_path, format='PNG')

In [ ]:
main()

 19%|█▉        | 115/610 [00:41<02:36,  3.17it/s]

 27%|██▋       | 164/610 [00:57<02:10,  3.41it/s]

 50%|█████     | 306/610 [01:47<01:42,  2.98it/s]

 55%|█████▍    | 335/610 [01:57<01:49,  2.52it/s]

 64%|██████▎   | 388/610 [02:15<01:18,  2.82it/s]

...

 15%|█▍        | 91/610 [00:43<03:02,  2.84it/s]

 35%|███▍      | 211/610 [01:23<01:19,  5.04it/s]

 74%|███████▍  | 450/610 [02:38<00:47,  3.38it/s]

 97%|█████████▋| 593/610 [03:25<00:03,  5.04it/s]

100%|██████████| 610/610 [03:32<00:00,  2.88it/s]


In [ ]:
source_dir = "/content/HRSC-YOLO/"
dest_dir = "/content/experimentals/"

# Create the 'experimentals' directory with 'images' and 'labels' subdirectories
os.makedirs('experimentals', exist_ok=True)
os.makedirs(os.path.join(dest_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(dest_dir, "labels"), exist_ok=True)

# Define dataset splits
splits = ['train', 'val', 'test']

# Copy all images and labels from HRSC-YOLO to experimentals
def copy_files():
    for split in splits:
        image_source = os.path.join(source_dir, split, "images")
        label_source = os.path.join(source_dir, split, "labels")

        for file in os.listdir(image_source):
            src_path = os.path.join(image_source, file)
            dest_path = os.path.join(dest_dir, "images", file)
            shutil.copy2(src_path, dest_path)

        for file in os.listdir(label_source):
            src_path = os.path.join(label_source, file)
            dest_path = os.path.join(dest_dir, "labels", file)
            shutil.copy2(src_path, dest_path)

copy_files()
print("All images and labels copied successfully to experimentals/")

All images and labels copied successfully to experimentals/


In [ ]:
# Define paths
base_dir = "/content/experimentals"
images_dir = os.path.join(base_dir, "images")
labels_dir = os.path.join(base_dir, "labels")

# Output directories
output_dirs = {
    "train": "train",
    "val": "val",
    "test": "test"
}

# Create train, val, and test directories with images and labels subfolders
for split in output_dirs.values():
    os.makedirs(os.path.join(split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split, "labels"), exist_ok=True)

# Get all image filenames
image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]
random.shuffle(image_files)  # Shuffle dataset

# Define split sizes
train_size, val_size, test_size = 610, 460, 610

# Split dataset
train_images = image_files[:train_size]
val_images = image_files[train_size:train_size + val_size]
test_images = image_files[train_size + val_size:train_size + val_size + test_size]

# Function to copy files
def copy_files(image_list, split):
    for img_name in image_list:
        label_name = img_name.replace(".png", ".txt")  # Corresponding label
        shutil.copy(os.path.join(images_dir, img_name), os.path.join(split, "images", img_name))
        shutil.copy(os.path.join(labels_dir, label_name), os.path.join(split, "labels", label_name))

# Copy files to respective folders
copy_files(train_images, "train")
copy_files(val_images, "val")
copy_files(test_images, "test")

print("Dataset split and copied successfully!")

Dataset split and copied successfully!


In [ ]:
yolo_dir2 = "/content"

In [ ]:
data_yaml = os.path.join(yolo_dir2, "data.yaml")

In [ ]:
def generate_data_yaml():
    yaml_content = f"""
train: {os.path.join(yolo_dir2, 'train/images')}
val: {os.path.join(yolo_dir2, 'val/images')}
test: {os.path.join(yolo_dir2, 'test/images')}

nc: 1
names: ['ship']
"""
    with open(data_yaml, "w") as f:
        f.write(yaml_content)
    print("Generated corrected data.yaml file.")

In [ ]:
generate_data_yaml()

Generated corrected data.yaml file.


In [ ]:
# download the YOLOv11-large model weights file from https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11l.pt and upload to the colab session

yolov11_model_path = "/content/yolo11l.pt"

In [ ]:
def train_yolov11(epochs=100, batch_size=8):
    model = YOLO(yolov11_model_path)
    model.train(data=data_yaml, epochs=epochs, batch=batch_size, imgsz=640, device='cuda', workers=4, save=True, save_period=10)
    model.val()
    print(model)
    print("YOLOv12 training completed on HRSC2016-MS.")

In [ ]:
train_yolov11()

Ultralytics 8.3.83 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo11l.pt, data=/content/data.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=10, cache=False, device=cuda, workers=4, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

train: Scanning /content/train/labels.cache... 610 images, 13 backgrounds, 0 corrupt: 100%|██████████| 610/610 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/val/labels.cache... 460 images, 6 backgrounds, 0 corrupt: 100%|██████████| 460/460 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 167 weight(decay=0.0), 174 weight(decay=0.0005), 173 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100       8.8G      1.374      3.102       1.46         83        640:  13%|█▎        | 10/77 [00:06<00:43,  1.56it/s]


KeyboardInterrupt: 

In [ ]:
def infer_yolov11(image_path, model_weights="path_to_the_saved_model_weights_(can_be_found_at_runs/detect/trainx/weights/best.pt"):
    model = YOLO(model_weights)
    results = model(image_path)
    for result in results:
        img = cv2.imread(image_path)
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            cls = int(box.cls[0])
            label = f"{model.names[cls]} {conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.show()

In [ ]:
test_image = "path_to_the_image_file"
infer_yolov11(test_image)

In [ ]:
# calculate mAP scores for the validation set

from ultralytics import YOLO

model = YOLO("path_to_the_saved_model_weights_(can_be_found_at_runs/detect/trainx/weights/best.pt")

metrics = model.val(data=data_yaml)
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)

In [ ]:
# over-write data.yaml and exchange the paths assigned to test and valid with each other
# calculate mAP scores for the test set

from ultralytics import YOLO

model = YOLO("path_to_the_saved_model_weights_(can_be_found_at_runs/detect/trainx/weights/best.pt")

metrics = model.val(data=data_yaml)
print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)